In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
pwd

In [2]:
train = pd.read_csv('./FraudTransaction/train.csv', nrows=100000)
#test = pd.read_csv('./FraudTransaction/test.csv')

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.info()

In [ ]:
sns.distplot(train['target'], kde=False)

In [ ]:
train.head()

In [16]:
train.columns

Index(['transaction_id', 'num_var_1', 'num_var_2', 'num_var_3', 'num_var_4',
       'num_var_5', 'num_var_6', 'num_var_7', 'cat_var_1', 'cat_var_2',
       'cat_var_3', 'cat_var_4', 'cat_var_5', 'cat_var_6', 'cat_var_7',
       'cat_var_8', 'cat_var_9', 'cat_var_10', 'cat_var_11', 'cat_var_12',
       'cat_var_13', 'cat_var_14', 'cat_var_15', 'cat_var_16', 'cat_var_17',
       'cat_var_18', 'cat_var_19', 'cat_var_20', 'cat_var_21', 'cat_var_22',
       'cat_var_23', 'cat_var_24', 'cat_var_25', 'cat_var_26', 'cat_var_27',
       'cat_var_28', 'cat_var_29', 'cat_var_30', 'cat_var_31', 'cat_var_32',
       'cat_var_33', 'cat_var_34', 'cat_var_35', 'cat_var_36', 'cat_var_37',
       'cat_var_38', 'cat_var_39', 'cat_var_40', 'cat_var_41', 'cat_var_42',
       'target'],
      dtype='object')

In [ ]:
#Missing Data
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

In [3]:
#cat_var_1 -- gf
#cat_var_3 -- qt
#cat_var_8 -- dn
train['cat_var_1'].fillna(value='gf',inplace=True)
train['cat_var_3'].fillna(value='qt',inplace=True)
train['cat_var_8'].fillna(value='dn',inplace=True)

In [13]:
count_class_0, count_class_1 = train.target.value_counts()
print(count_class_0, count_class_1)
print(count_class_1/count_class_0)

89317 35726
0.399991043138


In [14]:
(count_class_0*2)/5

35726.800000000003

In [5]:
#Resample the data
# Class count
count_class_0, count_class_1 = train.target.value_counts()

# Divide by class
df_class_0 = train[train['target'] == 0]
df_class_1 = train[train['target'] == 1]

df_class_1_over = df_class_1.sample(int((count_class_0*2)/5), replace=True)

train_over = pd.concat([df_class_0, df_class_1_over], axis=0)

# Shuffle data
idx = np.arange(len(train_over))
np.random.shuffle(idx)
train_over = train_over.iloc[idx]

train = train_over

print(train.shape)

(125043, 51)


In [19]:
cat_cols = [x for x in train.columns if x.startswith('cat')]

In [21]:
#cat_cols

In [6]:
train_features = [x for x in train.columns if x not in ['transaction_id','target']]

In [7]:
#Encode the data
for column in train_features:
    if train[column].dtype == type(object):
        le = LabelEncoder()
        le.fit(train[column])
        train[column] = le.transform(train[column])

In [22]:
df_with_dummies = pd.get_dummies(train, columns = cat_cols)

In [27]:
#df_with_dummies['cat_var_1']

In [28]:
X = train[train_features].values
y = train.loc[:,'target'].values
#T = test[train_features].values

In [29]:
y

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [30]:
stacker = RandomForestClassifier(n_estimators=100,random_state=0)
results = cross_val_score(stacker, X, y, cv=5, scoring='roc_auc')
print(results)
print("Stacker score: {} ".format(results.mean()))

[ 0.93021103  0.92769552  0.93053278  0.92984304  0.93037961]
Stacker score: 0.9297323940086073 


In [11]:
stacker = XGBClassifier(n_estimators=150,seed=0)
results = cross_val_score(stacker, X, y, cv=5, scoring='roc_auc')
print(results)
print("Stacker score: {} for num: ".format(results.mean()))

[ 0.72867825  0.73225522  0.73645648  0.74186676  0.73841452]
Stacker score: 0.7355342464391806 for num: 


In [ ]:
#Lets check the feature importance using XGB
features = train_features

xgb_params = {
    'n_estimators':200,
    'eta': 0.05,
    'max_depth': 4,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'binary:logistic',
    'min_child_weight':1,
    'silent': 1,
    'seed':0,
    'eval_metric':'auc'
}

x_Train, x_Test, y_Train, y_Test = train_test_split(X, y, test_size = 0.25, random_state = 0)

xgtrain = xgb.DMatrix(x_Train,y_Train, feature_names=features)
xgtest = xgb.DMatrix(x_Test,y_Test, feature_names=features)
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
num_rounds = 100 # Increase the number of rounds while running in local
model = xgb.train(xgb_params, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=5)

# plot the important features #
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()

In [ ]:
gb = RandomForestClassifier(n_estimators=200, max_depth=3, min_samples_leaf=4, max_features=0.2, random_state=0)
gb.fit(train[train_features], train.target)
features = train[train_features].columns.values
print("----- Training Done -----")

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(gb.feature_importances_, features), 
                                                            reverse = False)))
feature_imp = pd.DataFrame(features)
feature_imp.columns = ['feature']
feature_imp['imp'] = gb.feature_importances_

In [ ]:
feature_imp.sort_values(by='imp',ascending=False)

In [ ]:
list(feature_imp.sort_values(by='imp',ascending=False)['feature'])

In [ ]:
features = ['num_var_4', 'cat_var_14', 'cat_var_2', 'cat_var_13', 'num_var_7', 'num_var_2', 'cat_var_15', 'cat_var_17',
             'num_var_6', 'cat_var_18', 'cat_var_6', 'cat_var_12', 'num_var_1', 'cat_var_4', 'cat_var_20', 'cat_var_19',
             'num_var_5', 'cat_var_24', 'cat_var_21', 'cat_var_5', 'cat_var_7', 'cat_var_22', 'cat_var_16', 'cat_var_26',
            'cat_var_9', 'cat_var_30', 'cat_var_29']

In [ ]:
X = train[features].values
y = train.loc[:,'target'].values

In [ ]:
stacker = RandomForestClassifier(n_estimators=150,random_state=0)
results = cross_val_score(stacker, X, y, cv=5, scoring='roc_auc')
print(results)
print("Stacker score: {} for num: ".format(results.mean()))

In [ ]:
stacker = XGBClassifier(n_estimators=350,seed=0)
results = cross_val_score(stacker, X, y, cv=5, scoring='roc_auc')
print(results)
print("Stacker score: {} for num: ".format(results.mean()))

In [ ]:
train.columns

In [ ]:
train.nunique()

In [ ]:
sns.distplot(train['cat_var_2'], kde=False)

In [ ]:
sns.jointplot(x='num_var_4',y='target',data=train)

In [ ]:
train['cat_var_1'].sort_values().unique()

In [ ]:
#cat_var_1 -- gf
#cat_var_3 -- qt
#cat_var_8 -- dn
train['cat_var_8'].value_counts()

In [ ]:
train.sum()

In [ ]:
train['cat_var_37'].value_counts()

In [ ]:
pca = PCA(n_components=2)

from sklearn import preprocessing
data_scaled = pd.DataFrame(preprocessing.scale(X),columns = features) 

pca.fit(X)

#The amount of variance that each PC explains
var = pca.explained_variance_ratio_

#Cumulative Variance explains
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)